# Converting a Scikit model into ONNX format

We'll start by training a scikit-learn model and demonstrate how we can convert this into ONNX format. In this example, we are using the make_classification to create a classification dataset and then train an MLP classifier, after being applying StandarScaler transform on the data.

First we install the required Python packages from requirements.txt.

In [ ]:
!pip install -r requirements.txt

### Import the necessary packages.

In [175]:
import timeit
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from skl2onnx.convert import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from onnxruntime import InferenceSession
from onnxmltools.utils import save_model

Generate dataset with 10,000 samples, having 10 features and 5 classes.

In [77]:
X_data, y_data = make_classification(n_samples=10000, n_classes=5, n_features=10, random_state=42, n_informative=7)

Split the dataset into training set and test set.

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X_data.astype(np.float32), y_data)

### Scikit Pipeline
We create a scikit pipeline, which standardises the data using StandardScaler() and then uses an MLPClassifier() to train the model.

In [88]:
model = Pipeline([('scaler', StandardScaler()), ('predictor', MLPClassifier(random_state=42))])
model.fit(X_train, y_train)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('predictor',
                 MLPClassifier(activation='relu', alpha=0.0001,
                               batch_size='auto', beta_1=0.9, beta_2=0.999,
                               early_stopping=False, epsilon=1e-08,
                               hidden_layer_sizes=(100,),
                               learning_rate='constant',
                               learning_rate_init=0.001, max_iter=200,
                               momentum=0.9, n_iter_no_change=10,
                               nesterovs_momentum=True, power_t=0.5,
                               random_state=42, shuffle=True, solver='adam',
                               tol=0.0001, validation_fraction=0.1,
                               verbose=False, warm_start=False))],
         verbose=False)

### Model accuracy
Calculate the accuracy of our model on the test set.

In [89]:
np.mean(model.predict(X_test) == y_test)

0.8528

### Conversion to ONNX
Convert the scikit model into ONNX format using convert_sklearn(), then save the ONNX model.

In [90]:
model_onnx = convert_sklearn(model, 'classification model', [('input', FloatTensorType([None, X_test.shape[1]]))])
save_model(model_onnx, 'mlp.onnx')

Visualise the onnx model using Netron: https://lutzroeder.github.io/netron/

### Load the onnx model
For inferening, we first load the model as shown below.

In [91]:
sess = InferenceSession('mlp.onnx')

### Prediction using onnxruntime
In order to run prediction on a test set, we call run() passing the test set like this:

In [92]:
res = sess.run(None, input_feed={'input': X_test})

The above function call returns two outputs: label(output 0) and class probability scores(output 1).

## Comparing results of onnx and Scikit models
Here, we compare the labels returned by onnxruntime with the labels predicted by scikit.

In [93]:
np.mean(res[0] == model.predict(X_test))

1.0

We can also match the predicted probability scores of the two models.

In [94]:
np.mean(np.isclose(list(map(lambda x: [x[0], x[1], x[2], x[3], x[4]], res[1])),
                   model.predict_proba(X_test), atol=1e-5))

1.0

### Benchmarking
We compare the prediction runtime of ONNX and scikit models.

In [132]:
def onnx_test(x_test):
    sess.run(None, input_feed={'input': x_test})

In [122]:
def scikit_test(x_test):
    x = model.predict_proba(x_test)
    np.argmax(x)

In [156]:
onnx_time, scikit_time = [], []
for batch in range(12):
    onnx_time.append(timeit.timeit(f'onnx_test(X_test[:{2 ** batch}])', globals=globals(), number=100) / 100)
    scikit_time.append(timeit.timeit(f'scikit_test(X_test[:{2 ** batch}])',  globals=globals(), number=100) / 100)

In [177]:
pd.DataFrame(np.array([onnx_time, scikit_time]).transpose() * 1000, columns=['onnxruntime(ms)', 'scikit(ms)'], 
             index=[2**i for i in range(12)])

,onnxruntime(ms),scikit(ms)
1,0.053951,0.329427
2,0.037674,0.267644
4,0.039859,0.270791
8,0.043920,0.275846
16,0.055212,0.280654
32,0.086143,0.320520
64,0.113393,0.376533
128,0.257239,0.481376
256,0.495993,0.748602
512,0.938491,1.144875


As shown in the table above, onnxruntime clearly outperforms scikit in terms of prediction runtime for various test batch sizes.